In [1]:
%%time
import cx_Oracle
import numpy as np
import pandas as pd

#고객별 2014년 2015년 총구매액 및 평균구매액 출력
conn=cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = conn.cursor()

Wall time: 1.69 s


In [221]:
df_one = pd.read_csv('기존고객등급횟수(진짜).csv',encoding='cp949')

In [222]:
df_one.columns

Index(['고객번호', '1기구매횟수', '2기구매횟수', '3기구매횟수', '4기구매횟수', '5기구매횟수', '6기구매횟수',
       '1기구매등급', '2기구매등급', '3기구매등급', '4기구매등급', '5기구매등급', '6기구매등급'],
      dtype='object')

In [223]:
df_one = df_one[['고객번호']]

In [224]:
# 기존고객
df_one

,고객번호
0,1
1,2
2,3
3,4
4,6
...,...
18873,19333
18874,19334
18875,19335
18876,19336


In [72]:
# 기존고객 고객번호만 따로 csv 하나 만듬
import csv
df_one.to_csv('고객번호만.csv',mode="w",encoding='euc-kr',index=False)

In [225]:
# A제휴사 1-6분기 객단가
cursor.execute('select * from real_A2')
x = cursor.fetchall()

df = pd.DataFrame(x, columns=['고객번호','A객단가'])
df

,고객번호,A객단가
0,00001,93895
1,00002,90160
2,00003,82407
3,00004,24003
4,00005,17410
...,...,...
18441,19377,36516
18442,19378,41619
18443,19379,63500
18444,19381,164902


In [226]:
# B제휴사 1-6분기 객단가
cursor.execute('select * from real_B2')
x = cursor.fetchall()

df1 = pd.DataFrame(x, columns=['고객번호','B객단가'])
df1

,고객번호,B객단가
0,00001,10515
1,00002,12970
2,00003,3932
3,00004,9264
4,00005,15745
...,...,...
17990,19378,10035
17991,19379,11297
17992,19380,16236
17993,19381,7088


In [227]:
# C제휴사 1-6분기 객단가
cursor.execute('select * from real_C2')
x = cursor.fetchall()

df2 = pd.DataFrame(x, columns=['고객번호','C객단가'])
df2

,고객번호,C객단가
0,00001,8548
1,00003,2587
2,00004,5222
3,00006,13236
4,00009,8428
...,...,...
15705,19373,1404
15706,19374,4935
15707,19376,1583
15708,19378,6376


In [228]:
# D제휴사 1-6분기 객단가
cursor.execute('select * from real_D2')
x = cursor.fetchall()

df3 = pd.DataFrame(x, columns=['고객번호','D객단가'])
df3

,고객번호,D객단가
0,00002,12329
1,00005,4560
2,00010,12868
3,00012,1693
4,00026,17699
...,...,...
2622,19375,10616
2623,19377,12855
2624,19379,7493
2625,19380,11017


In [230]:
df_one.dtypes

고객번호    int64
dtype: object

In [234]:
# sql에서 가지고 온 데이터프레임이 int32여서 int64로 변경
df = df.astype('int64')

In [237]:
df1 = df1.astype('int64')

In [238]:
df2 = df2.astype('int64')

In [239]:
df3 = df3.astype('int64')

In [276]:
# 데이터프레임 좌측 기준으로 결합 
df_two = pd.merge(df_one,df,on='고객번호',how ='left')

In [277]:
df_three = pd.merge(df_two,df1,on='고객번호',how ='left')

In [278]:
df_four = pd.merge(df_three,df2,on='고객번호',how ='left')

In [281]:
df_five = pd.merge(df_four,df3,on='고객번호',how ='left')

In [283]:
df_five = df_five.fillna(0)

In [284]:
df_five2 = df_five.copy()

In [262]:
df_five2 =df_five2.set_index('고객번호')

In [263]:
# 이름바꾸기 사실 안해도 될것같은데 귀찮아서..
df_five2.rename(columns={'A객단가':'14_1','B객단가':'14_2','C객단가':'14_3','D객단가':'14_4'},inplace=True)
df_five2.head()

,14_1,14_2,14_3,14_4
고객번호,,,,
1,93895.0,10515.0,8548.0,0.0
2,90160.0,12970.0,0.0,12329.0
3,82407.0,3932.0,2587.0,0.0
4,24003.0,9264.0,5222.0,0.0
6,43597.0,6887.0,13236.0,0.0


In [265]:
df_five2 = df_five2.drop(['14_3','14_4'],axis=1)

In [266]:
aa=df_five2['14_1'].sort_values(ascending=False)
cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
cc=pd.concat([aa,cuts],axis=1)
cc

,14_1,14_1
고객번호,,
14323,33221393.0,1
15999,5116771.0,1
14599,4666471.0,1
8079,3079060.0,1
16406,2835301.0,1
...,...,...
14784,0.0,10
8961,0.0,10
12163,0.0,10


In [267]:
lst=['14_2']
for i in lst:
    aa=df_five2[f'{i}'].sort_values(ascending=False)
    
    cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
    table=pd.concat([aa,cuts],axis=1)  
    cc=pd.merge(cc,table, on='고객번호',how='left')
    
cc

,14_1,14_1,14_2,14_2
고객번호,,,,
14323,33221393.0,1,5700.0,8
15999,5116771.0,1,9657.0,4
14599,4666471.0,1,12774.0,2
8079,3079060.0,1,16415.0,2
16406,2835301.0,1,7480.0,6
...,...,...,...,...
14784,0.0,10,8440.0,5
8961,0.0,10,8070.0,5
12163,0.0,10,0.0,10


In [268]:
cc.columns=['A객단가','A객단가등급','B객단가','B객단가등급']

In [269]:
# 인덱스에 들어간 고객번호 컬럼으로 변경
cc1 = cc.reset_index()

In [270]:
alpha = pd.merge(df_one,cc1,on='고객번호')

In [271]:
alpha

,고객번호,A객단가,A객단가등급,B객단가,B객단가등급
0,1,93895.0,3,10515.0,3
1,2,90160.0,4,12970.0,2
2,3,82407.0,4,3932.0,10
3,4,24003.0,9,9264.0,4
4,6,43597.0,7,6887.0,7
...,...,...,...,...,...
18873,19333,6493.0,10,3649.0,10
18874,19334,0.0,10,6611.0,7
18875,19335,129331.0,2,9682.0,4
18876,19336,58193.0,5,0.0,10


In [272]:
alpha2 = alpha[['고객번호','A객단가등급','B객단가등급']]

In [273]:
alpha2.to_csv('기존고객AB객단가등급(진짜).csv',mode="w",encoding='euc-kr',index=False)

# 비율 구하기

In [247]:
df_five3 = df_five.copy()

In [248]:
# 고객번호 인덱스화
df_five3 = df_five3.set_index('고객번호')

In [275]:
# 결과
df_five3

,14_1,14_2
고객번호,,
1,93895.0,10515.0
2,90160.0,12970.0
3,82407.0,3932.0
4,24003.0,9264.0
6,43597.0,6887.0
...,...,...
19333,6493.0,3649.0
19334,0.0,6611.0
19335,129331.0,9682.0


In [250]:
# 비율구하기 공식
df_five3 = df_five3.div(df_five2.sum(axis=1),axis=0)

In [251]:
# 한번더 null값을 0 으로채움 급해서 나도 지금 fillna 왜 두번했는지 모름
df_five3 = df_five3.fillna(0)

In [252]:
# 데이터 프레임 전체에 100을 곱함
df_five3 = df_five3*100

In [253]:
# 데이터 프레임 전체에 소수점 2자리까지만 출력
df_five3 = df_five3.round(2)

In [254]:
# 결과
df_five3

,A객단가,B객단가,C객단가,D객단가
고객번호,,,,
1,83.12,9.31,7.57,0.00
2,78.09,11.23,0.00,10.68
3,92.67,4.42,2.91,0.00
4,62.36,24.07,13.57,0.00
6,68.42,10.81,20.77,0.00
...,...,...,...,...
19333,27.06,15.21,6.99,50.75
19334,0.00,71.19,28.81,0.00
19335,90.31,6.76,2.93,0.00


In [255]:
# 인덱스에 들어간 고객번호를 다시 올림
df_zero = df_five3.reset_index()

In [ ]:
# 고객번호 다시 올렸을 떄 나오는 결과값
# df_zero

In [258]:
# 컬럼명 전체 변경
df_zero.columns = ['고객번호','A제휴사(%)','B제휴사(%)','C제휴사(%)','D제휴사(%)']

In [260]:
# 결과
df_zero

,고객번호,A제휴사(%),B제휴사(%),C제휴사(%),D제휴사(%)
0,1,83.12,9.31,7.57,0.00
1,2,78.09,11.23,0.00,10.68
2,3,92.67,4.42,2.91,0.00
3,4,62.36,24.07,13.57,0.00
4,6,68.42,10.81,20.77,0.00
...,...,...,...,...,...
18873,19333,27.06,15.21,6.99,50.75
18874,19334,0.00,71.19,28.81,0.00
18875,19335,90.31,6.76,2.93,0.00
18876,19336,86.71,0.00,0.00,13.29


In [261]:
# 인코딩 + 쓰레기값 없이 저장
import csv
df_zero.to_csv('제휴사비율.csv',mode="w",encoding='euc-kr',index=False)